In [ ]:
from collections import defaultdict
from typing import Literal

from pydantic import BaseModel

In [ ]:
MAZE = dict()
with open("day16_input.txt") as file:
    for row, line in enumerate(file):
        for col, char in enumerate(line.strip()):
            if char in "/\\|-":
                MAZE[(row, col)] = char

MAX_ROWS = row
MAX_COLS = col

In [ ]:
class Beam(BaseModel):
    pos: tuple[int, int]
    dir: Literal["up", "down", "left", "right"]

    def next_pos(self, max_rows, max_cols) -> tuple[int, int] | None:
        row, col = self.pos
        match self.dir:
            case "up":
                pos = (row - 1, col)
            case "down":
                pos = (row + 1, col)
            case "left":
                pos = (row, col - 1)
            case "right":
                pos = (row, col + 1)

        row, col = pos
        if (0 <= row <= max_rows) and (0 <= col <= max_cols):
            return pos

    def __repr__(self):
        return f"Beam({self.pos}, {self.dir})"

In [ ]:
def find_num_energized(start: Beam) -> int:
    energized = defaultdict(set[str])
    beams = [start]

    while beams:
        beam = beams.pop()
        while True:
            if beam.dir in energized[beam.pos]:
                # A beam has already been here, moving in this direction.
                # This beam is done.
                break

            # Mark this position as visited in this direction.
            energized[beam.pos].add(beam.dir)

            # What is the next position?
            next_pos = beam.next_pos(MAX_ROWS, MAX_COLS)
            if next_pos is None:
                # This beam has left the maze, and is done.
                break

            # What to do next?
            char = MAZE.get(next_pos)

            # The beam is being split
            if (char == "|") and (beam.dir in ("left", "right")):
                # Add two new beams
                beams.append(Beam(pos=next_pos, dir="up"))
                beams.append(Beam(pos=next_pos, dir="down"))
                # This beam is done
                break
            elif (char == "-") and (beam.dir in ("up", "down")):
                # Add two new beams
                beams.append(Beam(pos=next_pos, dir="left"))
                beams.append(Beam(pos=next_pos, dir="right"))
                # This beam is done
                break

            # The beam is changing direction
            elif char == "/":
                if beam.dir == "right":
                    beam.dir = "up"
                elif beam.dir == "left":
                    beam.dir = "down"
                elif beam.dir == "up":
                    beam.dir = "right"
                elif beam.dir == "down":
                    beam.dir = "left"
            elif char == "\\":
                if beam.dir == "right":
                    beam.dir = "down"
                elif beam.dir == "left":
                    beam.dir = "up"
                elif beam.dir == "up":
                    beam.dir = "left"
                elif beam.dir == "down":
                    beam.dir = "right"

            # Move the beam to the new position, and continue.
            beam.pos = next_pos

    # Subtract 1 because the starting position is outside the grid.
    return len(energized) - 1

# Part 1


In [ ]:
print("Answer:", find_num_energized(Beam(pos=(0, -1), dir="right")))

# Part 2


In [ ]:
# Brute force solution
energized = []
for row in range(MAX_ROWS + 1):
    start = Beam(pos=(row, -1), dir="right")
    energized.append(find_num_energized(start))
    start = Beam(pos=(row, MAX_COLS + 1), dir="left")
    energized.append(find_num_energized(start))

for col in range(MAX_COLS + 1):
    start = Beam(pos=(-1, col), dir="down")
    energized.append(find_num_energized(start))
    start = Beam(pos=(MAX_ROWS + 1, col), dir="up")
    energized.append(find_num_energized(start))

print("Answer:", max(energized))